# Homework 2 - Recurrent Neural Networks


In this part of the homework we are going to work with Recurrent Neural Networks, in particular GRU. One of the greatest things that Recurrent Neural Networks can do when working with sequences is retaining data from several timesteps in the past. We are going to explore that property by constructing an 'echo' Recurrent Neural Network.

The goal here is to make a model that given a sequence of letters or digits will output that same sequence, but with a certain delay. Let's say the input is a string 'abacaba', we want the model to not output anything for 3 steps (delay length), and then output the original string step by step, except the last 3 characters. So, target output is then 'XXXabac', where 'X' is empty output.

This is similar to [this notebook](https://github.com/Atcold/pytorch-Deep-Learning/blob/master/09-echo_data.ipynb) (which you should refer to when doing this assignment), except we're working not with a binary string, but with a sequence of integers between 0 and some N. In our case N is 26, which is the number of letters in the alphabet.


## Dataset

Let's implement the dataset. In our case, the data is basically infinite, as we can always generate more examples on the fly, so there's no need to load it from disk.


In [85]:
import random
import string

import torch

# Max value of the generated integer. 26 is chosen becuase it's
# the number of letters in English alphabet.
N = 26


def idx_to_onehot(x, k=N+1):
    """ Converts the generated integers to one-hot vectors """
    ones = torch.sparse.torch.eye(k)
    shape = x.shape
    res = ones.index_select(0, x.view(-1).type(torch.int64))
    return res.view(*shape, res.shape[-1])


class EchoDataset(torch.utils.data.IterableDataset):

    def __init__(self, delay=4, seq_length=15, size=1000):
        self.delay = delay
        self.seq_length = seq_length
        self.size = size

    def __len__(self):
        return self.size

    def __iter__(self):
        """ Iterable dataset doesn't have to implement __getitem__.
            Instead, we only need to implement __iter__ to return
            an iterator (or generator).
        """
        for _ in range(self.size):
            seq = torch.tensor([random.choice(range(1, N + 1))
                               for i in range(self.seq_length)], dtype=torch.int64)
            result = torch.cat(
                (torch.zeros(self.delay), seq[:self.seq_length - self.delay])).type(torch.int64)
            yield seq, result


DELAY = 4
DATASET_SIZE = 200000
ds = EchoDataset(delay=DELAY, size=DATASET_SIZE)


## Model

Now, we want to implement the model. For our purposes, we want to use GRU. The architecture consists of GRU and a decoder. Decoder is responsible for decoding the GRU hidden state to yield a predicting for the next output. The parts you are responsible for filling with your code are marked with `TODO`.


In [86]:
import numpy as np


class GRUMemory(torch.nn.Module):

    def __init__(self, input_size, hidden_size, num_layers, num_classes):
        super().__init__()
        # TODO: initialize your submodules

        self.num_layers = num_layers
        self.hidden_size = hidden_size
        self.gru = nn.GRU(input_size, hidden_size,
                          num_layers, batch_first=True)
        # x is [batch x seq x input]
        self.fc = nn.Linear(hidden_size, num_classes)
        self.relu = nn.ReLU()

    def forward(self, x):
        # inputs: x - input tensor of shape (batch_size, seq_length, N+1)
        # returns:
        # logits (scores for softmax) of shape (batch size, seq_length, N + 1)
        # TODO implement forward pass

        # Hidden state init for first input to 0; forward propagation by passing in the input and hidden state into the model
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size)
        out, _ = self.gru(x, h0)  # dont need hidden state
        # out: tensor of shape (batch_size, seq_length, hidden_size)
        # out (z, N+1, hidden size)

        # Decode the hidden state of the last time step
        out = out[:, -1, :]

        out = self.fc(self.relu(out))

        return out

    @torch.no_grad()
    def test_run(self, s):
        # This function accepts one string s containing lowercase characters a-z.
        # You need to map those characters to one-hot encodings,
        # then get the result from your network, and then convert the output
        # back to a string of the same length, with 0 mapped to ' ',
        # and 1-26 mapped to a-z.

        # Use ASCII tricks: https://www.asciitable.com/ where a is 97 to z being 122 and a space is 32
        alpha_to_num = torch.from_numpy(
            np.array([ord(i)-96 if ord(i) >= 97 else ord(i) - 32 for i in list(s)]))
        dis_fire = idx_to_onehot(alpha_to_num).unsqueeze(0)

        logits = self.forward(dis_fire)
        predict = torch.argmax(logits, dim=2)
        return [chr(i + 96) if i >= 1 else chr(i+32) for i in predict[0, :]]


## Training

Below you need to implement the training of the model. We give you more freedom as for the implementation. The two limitations are that it has to execute within 10 minutes, and that error rate should be below 1%.


In [87]:
def test_model(model, sequence_length=15):
    """
    This is the test function that runs 100 different strings through your model,
    and checks the error rate.
    """
    total = 0
    correct = 0
    for i in range(500):
        s = ''.join([random.choice(string.ascii_lowercase)
                    for i in range(random.randint(15, 25))])
        result = model.test_run(s)
        for c1, c2 in zip(s[:-DELAY], result[DELAY:]):
            correct += int(c1 == c2)
        total += len(s) - DELAY

    return correct / total


In [88]:
from torch.utils.data import random_split, DataLoader
import torch.nn as nn
import torch.nn.functional as F

MAX_EPOCHS = 10
BATCH_SIZE = 64

# train_count = int(0.7 * DATASET_SIZE)
# valid_count = int(0.2 * DATASET_SIZE)
# test_count = DATASET_SIZE - train_count - valid_count
# train_dataset, valid_dataset, test_dataset = torch.utils.data.random_split(
#     ??, (train_count, valid_count, test_count)
# )

train_loader = torch.utils.data.DataLoader(dataset=ds,
                                           batch_size=BATCH_SIZE)

test_loader = torch.utils.data.DataLoader(dataset=ds,
                                          batch_size=BATCH_SIZE,
                                          shuffle=False)


In [89]:
import time
# Device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Move model to the device which will be used for train and test
input_size = N + 1
sequence_len = N + 1
num_classes = sequence_len  # output size
num_layers = 2  # stck em up
hidden_size = 64

model = GRUMemory(input_size, hidden_size, num_layers, num_classes)
model.to(device)
criterion = torch.nn.CrossEntropyLoss()

if torch.cuda.is_available():
    model = model.cuda()
    criterion = criterion.cuda()
    device = torch.device("cuda:0")
else:
    device = torch.device("cpu")

optimizer = torch.optim.Adam(model.parameters(), lr=3e-4)

start_time = time.time()

# Train the model
n_total_steps = len(train_loader)
for epoch in range(MAX_EPOCHS):
    for i, (data, target) in enumerate(train_loader):  

        data = idx_to_onehot(data).to(device)
        target = target.to(device)
        
        # Forward pass
        outputs = model(data)
        loss = criterion(outputs, target)
        
        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if (i+1) % 100 == 0:
            print (f'Epoch [{epoch+1}/{MAX_EPOCHS}], Step [{i+1}/{n_total_steps}], Loss: {loss.item():.4f}')


end_time = time.time()
duration = end_time - start_time
accuracy = test_model(model)
assert duration < 600, 'execution took f{duration:.2f} seconds, which longer than 10 mins'
assert accuracy > 0.99, f'accuracy is too low, got {accuracy}, need 0.99'
print('tests passed')


RuntimeError: Input and hidden tensors are not at the same device, found input tensor at cuda:0 and hidden tensor at cpu

## Variable delay model

Now, to make this more complicated, we want to have varialbe delay. So, now, the goal is to transform a sequence of pairs (character, delay) into a character sequence with given delay. Delay is constant within one sequence.


### Dataset

As before, we first implement the dataset:


In [ ]:
class VariableDelayEchoDataset(torch.utils.data.IterableDataset):

    def __init__(self, max_delay=8, seq_length=20, size=1000):
        self.max_delay = max_delay
        self.seq_length = seq_length
        self.size = size

    def __len__(self):
        return self.size

    def __iter__(self):
        for _ in range(self.size):
            seq = torch.tensor([random.choice(range(1, N + 1))
                               for i in range(self.seq_length)], dtype=torch.int64)
            delay = random.randint(0, self.max_delay)
            result = torch.cat(
                (torch.zeros(delay), seq[:self.seq_length - delay])).type(torch.int64)
            yield seq, delay, result


### Model

And the model.


In [ ]:
class VariableDelayGRUMemory(torch.nn.Module):

    def __init__(self, hidden_size, max_delay):
        super().__init__()
        # TODO

    def forward(self, x, delays):
        # inputs:
        # x - tensor of shape (batch size, seq length, N + 1)
        # delays - tensor of shape (batch size)
        # returns:
        # logits (scores for softmax) of shape (batch size, seq_length, N + 1)

        # TODO
        pass

    @torch.no_grad()
    def test_run(self, s, delay):
        # This function accepts one string s containing lowercase characters a-z,
        # and a delay - the desired output delay.
        # You need to map those characters to one-hot encodings,
        # then get the result from your network, and then convert the output
        # back to a string of the same length, with 0 mapped to ' ',
        # and 1-26 mapped to a-z.

        # TODO
        pass


### Train

As before, you're free to do what you want, as long as training finishes within 10 minutes and accuracy is above 0.99 for delays between 0 and 8.


In [ ]:
def test_variable_delay_model(model, seq_length=20):
    """
    This is the test function that runs 100 different strings through your model,
    and checks the error rate.
    """
    total = 0
    correct = 0
    for i in range(500):
        s = ''.join([random.choice(string.ascii_lowercase)
                    for i in range(seq_length)])
        d = random.randint(0, model.max_delay)
        result = model.test_run(s, d)
        if d > 0:
            z = zip(s[:-d], result[d:])
        else:
            z = zip(s, result)
        for c1, c2 in z:
            correct += int(c1 == c2)
        total += len(s) - d

    return correct / total


In [ ]:
import time
start_time = time.time()

MAX_DELAY = 8
SEQ_LENGTH = 20

# TODO: implement model training here.
model = None


end_time = time.time()
assert end_time - start_time < 600, 'executing took longer than 10 mins'
assert test_variable_delay_model(model) > 0.99, 'accuracy is too low'
print('tests passed')
